In [1]:
import numpy as np
import geopandas as gpd
import pandas as pd
from miscelanious.trsfm_shp import gpd2pv
from miscelanious.splitRotate import rotate_matrix, pvtopandas

In [2]:
shp_files = [
    r"D:\00_Local_scale\coral_garden_22\02-reprojection\mtshp_export_no_offset\corals_EPSG32629_csv_corals_yolov8_PC.shp",
    r"D:\00_Local_scale\coral_garden_22\02-reprojection\mtshp_export_no_offset\AS_EPSG32629.shp"
]

export_file = r"D:\00_Local_scale\coral_garden_22\02-reprojection\EPSG32629_aligned_grouped\annotations_EPSG32629_aligned_011025.csv"

transform_matrix = np.array([
    [0.995613455772, -0.093462236226, -0.004322697408, 25.882957458496],
    [0.093302078545, 0.995228230953, -0.028557799757, -120.604492187500],
    [0.006971145980, 0.028029212728, 0.999582767487, -10.708086013794],
    [0.0, 0.0, 0.0, 1.0]
])

offset = [609000, 5274000]

In [4]:
annotations_coral = gpd.read_file(shp_files[0])
annotations_coral.set_crs('epsg:32629', inplace=True)
annotations_coral = annotations_coral.explode(ignore_index=True)
annotations_coral["NAME"] = "Coral"
annotations_AS = gpd.read_file(shp_files[1])
annotations_AS.set_crs('epsg:32629', inplace=True)

annotations = pd.concat([annotations_coral, annotations_AS], ignore_index=True)

annotations_pc = gpd2pv(annotations, offset)
annotations_pc['Classification'] = annotations.NAME

rotated_pc = rotate_matrix(annotations_pc, transform_matrix)
result = pvtopandas(rotated_pc)
result['x'] = result['x'] + offset[0]
result['y'] = result['y'] + offset[1]
result.to_csv(export_file, index=False)